## Détection automatique des villes dans les adresses + Longitude, latitude

#### ⚠️ Ne couvre que les villes de plus de **15 000 habitants**

In [1]:
nom_de_la_structure="Sophia"

In [ ]:
import pandas as pd

# Chargement du fichier
file_path = "cities15000/cities15000.txt"

columns = [
    "geonameid", "name", "asciiname", "alternatenames",
    "latitude", "longitude", "feature_class", "feature_code",
    "country_code", "cc2", "admin1_code", "admin2_code",
    "admin3_code", "admin4_code", "population", "elevation",
    "dem", "timezone", "modification_date"
]

df_cities = pd.read_csv(file_path, sep="\t", names=columns, dtype=str)

# --- Garder seulement les colonnes utiles ---
df_cities = df_cities[["name", "asciiname", "alternatenames", "country_code"]]

# --- Liste 1 : uniquement les 'name' ---
liste_names = df_cities["name"].dropna().unique().tolist()

# --- Liste 2 : 'name' + 'asciiname' + 'alternatenames' ---
expanded = []

for _, row in df_cities.iterrows():
    if pd.notna(row["asciiname"]):
        expanded.append(row["asciiname"])
    if pd.notna(row["alternatenames"]):
        expanded.extend(row["alternatenames"].split(","))

# Nettoyer doublons / vides
liste_all = sorted(set([x.strip() for x in expanded if x and x.strip()]))

print("✅ Nombre de villes officielles :", len(liste_names))
print("✅ Nombre de variantes (avec alternatenames) :", len(liste_all))

# Exemple d'aperçu
print("Exemples liste_names:", liste_names[:10])
print("Exemples liste_all:", liste_all[:20])


✅ Nombre de villes officielles : 30785
✅ Nombre de variantes (avec alternatenames) : 306013
Exemples liste_names: ['les Escaldes', 'Andorra la Vella', 'Warīsān', 'Umm Suqaym', 'Umm Al Quwain City', 'Ţarīf Kalbā', 'Ar Rāshidīyah', 'Ras Al Khaimah City', 'Zayed City', 'Khawr Fakkān']
Exemples liste_all: ["'Abasan al Kabirah", "'Afak", "'Afula", "'Ain Abid", "'Ain Azel", "'Ain Benian", "'Ain Deheb", "'Ain Merane", "'Ain al 'Awda", "'Ain el 'Aouda", "'Ain el Bell", "'Ain el Berd", "'Ain el Bia", "'Ain el Hammam", "'Ain el Melh", "'Ain el Turk", "'Ajlun", "'Akko", "'Alem Ketema", "'Alem T'ena"]


In [7]:
import unidecode
key = unidecode.unidecode("Ispra").lower()
if key in liste_all:
    print("✅ Trouvé :", liste_all[key])
else:
    print("❌ Pas trouvé")

❌ Pas trouvé


In [3]:
import pandas as pd
import re
import unidecode
from tqdm import tqdm

# ------------------ PARAMS ------------------
F_IN  = f"Copubliants_par_auteur_Inria_{nom_de_la_structure}_ville_final.xlsx"
COL_ADR = "Adresse"
COL_ORG = "Organisme copubliant"

tqdm.pandas()

STOPWORDS = {"la", "le", "de", "del", "da", "di", "el"}

# --- 1) lookup rapide: liste_names normalisée -> forme canonique ---
def _norm(s):
    return unidecode.unidecode(str(s)).lower().strip()

names_lookup = {}
for n in liste_names:
    key = _norm(n)
    if key and len(key) >= 3 and key not in STOPWORDS:
        # garde la 1re forme rencontrée comme canonique
        names_lookup.setdefault(key, n)

# --- 2) extraction ville via crochets [ ... ] uniquement contre liste_names ---
def ville_from_brackets(organisme: str):
    if pd.isna(organisme):
        return None
    m = re.search(r"\[(.*?)\]", str(organisme))
    if not m:
        return None
    inside = _norm(m.group(1))
    if len(inside) < 3 or inside in STOPWORDS:
        return None
    # match exact dans liste_names (normalisé)
    return names_lookup.get(inside, None)

# --- 3) détection via adresse (ton algo) ---
def trouver_ville(adresse, liste_names, liste_all):
    if pd.isna(adresse):
        return None
    
    texte = unidecode.unidecode(str(adresse)).lower().strip()
    if "," in texte:
        last_part = texte.split(",")[-1].strip()
    elif "-" in texte:
        last_part = texte.split("-")[-1].strip()
    else:
        last_part = " ".join(texte.split()[-2:])

    def match_in_list(segment, villes):
        for ville in villes:
            v = unidecode.unidecode(str(ville)).lower()
            if not v or v in STOPWORDS or len(v) < 3:
                continue
            if re.search(rf"\b{re.escape(v)}\b", segment):
                return ville
        return None

    # d'abord dernier segment, puis fallback texte entier
    for seg in (last_part, texte):
        v = match_in_list(seg, liste_names)
        if v:
            return v
        v = match_in_list(seg, liste_all)
        if v:
            return v
    return None

# --- 4) normalisation adresse pour dédoublonnage ---
def norm_addr(s):
    if pd.isna(s): 
        return ""
    s = unidecode.unidecode(str(s)).lower().strip()
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ------------------ PIPELINE ------------------
df = pd.read_excel(F_IN)

# A) tentative rapide via crochets (uniquement liste_names)
df["Ville"] = df[COL_ORG].progress_apply(ville_from_brackets)

# B) ne traiter par adresse que les lignes encore vides
mask_todo = df["Ville"].isna()

# Clé normalisée (pour dédoublonner) uniquement sur le sous-ensemble à traiter
df.loc[mask_todo, "_addr_norm"] = df.loc[mask_todo, COL_ADR].progress_apply(norm_addr)

# Adresses uniques à traiter
uniq = (
    df.loc[mask_todo & (df["_addr_norm"] != ""), [COL_ADR, "_addr_norm"]]
      .drop_duplicates("_addr_norm", keep="first")
      .reset_index(drop=True)
)

# Détection ville pour ces adresses uniques
if not uniq.empty:
    uniq["Ville_detectee_unique"] = uniq[COL_ADR].progress_apply(
        lambda x: trouver_ville(x, liste_names, liste_all)
    )
    addr2city = dict(zip(uniq["_addr_norm"], uniq["Ville_detectee_unique"]))
    # Re-propager sur le sous-ensemble
    df.loc[mask_todo, "Ville"] = df.loc[mask_todo, "_addr_norm"].map(addr2city)

# Nettoyage colonne technique si créée
if "_addr_norm" in df.columns:
    df.drop(columns=["_addr_norm"], inplace=True)


100%|██████████| 1196/1196 [1:29:50<00:00,  4.51s/it]


In [5]:
df.to_excel(f"Copubliants_par_auteur_Inria_{nom_de_la_structure}_2.xlsx")
print("✅ Fichier créé")

✅ Fichier créé


In [6]:
# Création des colonnes longitudes et lattitudes
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Charger le fichier
df = pd.read_excel(f"Copubliants_par_auteur_Inria_{nom_de_la_structure}_2.xlsx")

# Nettoyer les colonnes
df.columns = [str(c).strip().replace("\xa0","").replace(" ","_") for c in df.columns]

# Vérifier que la colonne Ville existe
if "Ville" not in df.columns:
    raise ValueError("Colonne 'Ville' non trouvée dans le fichier")

# Vérifier si la colonne Pays existe
has_country = "Pays" in df.columns

# Initialiser le géocodeur
geolocator = Nominatim(user_agent="inria_world_app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Créer un dictionnaire pour stocker les coordonnées
ville_coords = {}

# Récupérer les coordonnées pour chaque combinaison ville+pays
for idx, row in df[["Ville"] + (["Pays"] if has_country else [])].dropna().drop_duplicates().iterrows():
    ville = row["Ville"]
    pays = row["Pays"] if has_country else ""

    # Construire la requête (ville + pays si dispo)
    query = f"{ville}, {pays}" if pays else ville

    try:
        location = geocode(query)
        if location:
            ville_coords[(ville, pays)] = (location.latitude, location.longitude)
        else:
            ville_coords[(ville, pays)] = (None, None)
            print(f"Coordonnées non trouvées pour : {query}")
    except Exception as e:
        ville_coords[(ville, pays)] = (None, None)
        print(f"Erreur pour {query} : {e}")

# Ajouter les colonnes Longitude et Latitude
df["Latitude"] = df.apply(lambda x: ville_coords.get((x["Ville"], x["Pays"] if has_country else ""), (None,None))[0], axis=1)
df["Longitude"] = df.apply(lambda x: ville_coords.get((x["Ville"], x["Pays"] if has_country else ""), (None,None))[1], axis=1)

# Sauvegarder le fichier
df.to_excel(f"Copubliants_par_auteur_Inria_{nom_de_la_structure}_2.xlsx", index=False)
print("Fichier sauvegardé avec les colonnes Latitude et Longitude !")


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Sydney, Australia',), **{}).
Traceback (most recent call last):
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
     

Coordonnées non trouvées pour : FOR, Hungary


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Martin, United States',), **{}).
Traceback (most recent call last):
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\site-packages\urllib3\connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\http\client.py", line 1378, in getresponse
    response.begin()
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dadasilv\AppData\Local\anaconda3\Lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
 

Fichier sauvegardé avec les colonnes Latitude et Longitude !
